# Modelo de recomendación de películas
## Objetivo
En esta práctica entrenaremos un **modelo de recomendación de películas**. Los modelos de recomendación pueden usar el **filtrado colaborativo**; usando puntuaciones de usuarios con gustos similares para elegir la recomendación, sin embargo, nosotros solo tendremos el conjunto de datos de puntuaciones de un usuario, por lo que usaremos un **filtrado basado en contenido**; usaremos las características de las películas vistas por el usuario para recomendar otras películas con características similares.

Primero tenemos que preparar el conjunto de datos objeto, las películas:

## Curando el dataset
Para entrenar el modelo usaremos los conjuntos de datos de [IMDB](https://developer.imdb.com/non-commercial-datasets/), title.basics para obtener los títulos, fecha de salida y géneros de las películas y title.crew para los directores de las películas.

Filtramos el conjunto para eliminar todo lo que no sea películas y lo guardamos en un archivo csv más fácil de procesar.


In [ ]:
import pandas as pd

# Cargamos el dataset de IMDB
df = pd.read_csv("datasets/title.basics.tsv", sep ='\t')

# Cogemos solo las películas y guardamos en otro archivo
df = df[df['titleType'] == 'movie']
df.to_csv("datasets/titles_filtered.csv")
df.head(3)

Eliminamos columnas de datos innecesarios y consideramos irrelevantes aquellos registros sin fecha de salida ni género.

In [22]:
import pandas as pd

df = pd.read_csv("datasets/titles_filtered.csv")

# Eliminamos la primera columna, son índices
df = df.drop(df.columns[0], axis=1)

# Eliminamos columnas irrelevantes
df = df.drop(['endYear', 'isAdult', 'titleType'], axis=1)

# Eliminamos películas sin fecha de salida ni géneros
df = df[df['startYear'] != '\\N']
df = df[df['genres'] != '\\N']

# Unificamos los títulos, todos en mayúscula
df['primaryTitle'] = df['primaryTitle'].str.upper()
df['originalTitle'] = df['originalTitle'].str.upper()

# Cambiamos los valores nulos de la duración a 0
df.loc[df['runtimeMinutes'] == '\\N', 'runtimeMinutes'] = 0

# Forzamos el tipo de startYear y runtimeMinutes a ser numérico,
# era mixto por los nulos ('\N')
df['startYear'] = pd.to_numeric(df['startYear'], downcast='integer', errors='coerce')
df['runtimeMinutes'] = pd.to_numeric(df['runtimeMinutes'], downcast='integer', errors='coerce')

df.head(10)

C:\Users\GUILLERMO\AppData\Local\Temp\ipykernel_6724\2247269571.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/titles_filtered.csv")


,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt0000009,MISS JERRY,MISS JERRY,1894,45,Romance
1,tt0000147,THE CORBETT-FITZSIMMONS FIGHT,THE CORBETT-FITZSIMMONS FIGHT,1897,100,"Documentary,News,Sport"
3,tt0000574,THE STORY OF THE KELLY GANG,THE STORY OF THE KELLY GANG,1906,70,"Action,Adventure,Biography"
4,tt0000591,THE PRODIGAL SON,L'ENFANT PRODIGUE,1907,90,Drama
5,tt0000615,ROBBERY UNDER ARMS,ROBBERY UNDER ARMS,1907,0,Drama
6,tt0000630,HAMLET,AMLETO,1908,0,Drama
7,tt0000675,DON QUIJOTE,DON QUIJOTE,1908,0,Drama
8,tt0000679,THE FAIRYLOGUE AND RADIO-PLAYS,THE FAIRYLOGUE AND RADIO-PLAYS,1908,120,"Adventure,Fantasy"
19,tt0000886,"HAMLET, PRINCE OF DENMARK",HAMLET,1910,0,Drama
20,tt0000941,LOCURA DE AMOR,LOCURA DE AMOR,1909,45,Drama


Unimos a nuestro conjunto los directores de cada película a partir del archivo de IMDB.

In [23]:
df_crew = pd.read_csv("datasets/title.crew.tsv", sep ='\t')

df = pd.merge(df, df_crew, on="tconst")

df.head(10)

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,directors,writers
0,tt0000009,MISS JERRY,MISS JERRY,1894,45,Romance,nm0085156,nm0085156
1,tt0000147,THE CORBETT-FITZSIMMONS FIGHT,THE CORBETT-FITZSIMMONS FIGHT,1897,100,"Documentary,News,Sport",nm0714557,\N
2,tt0000574,THE STORY OF THE KELLY GANG,THE STORY OF THE KELLY GANG,1906,70,"Action,Adventure,Biography",nm0846879,nm0846879
3,tt0000591,THE PRODIGAL SON,L'ENFANT PRODIGUE,1907,90,Drama,nm0141150,nm0141150
4,tt0000615,ROBBERY UNDER ARMS,ROBBERY UNDER ARMS,1907,0,Drama,nm0533958,"nm0092809,nm0533958"
5,tt0000630,HAMLET,AMLETO,1908,0,Drama,nm0143333,nm0000636
6,tt0000675,DON QUIJOTE,DON QUIJOTE,1908,0,Drama,nm0194088,nm0148859
7,tt0000679,THE FAIRYLOGUE AND RADIO-PLAYS,THE FAIRYLOGUE AND RADIO-PLAYS,1908,120,"Adventure,Fantasy","nm0091767,nm0877783","nm0000875,nm0877783"
8,tt0000886,"HAMLET, PRINCE OF DENMARK",HAMLET,1910,0,Drama,nm0099901,nm0000636
9,tt0000941,LOCURA DE AMOR,LOCURA DE AMOR,1909,45,Drama,"nm0063413,nm0550220","nm0063413,nm0550220,nm0848502"


Comprobamos la integridad de los tipos de las columnas.

In [24]:
for column in df.columns:
    print(column, ":", pd.api.types.infer_dtype(df[column]))

tconst : string
primaryTitle : string
originalTitle : string
startYear : integer
runtimeMinutes : integer
genres : string
directors : string
writers : string


Guardamos un csv nuevo con el nuevo registro con los datos que usaremos para el entrenamiento.

In [26]:
df.to_csv("datasets/movies_curated.csv")

## Filtrado basado en contenido

Como hemos dicho previamente, es un sistema de filtrado que nos permite usar las características (género y cast) de las películas vistas por el usuario para recomendar películas similares.

Este sistema funciona ubicando las películas en un espacio vectorial dependiendo de su vector de características y comparándolas con el vector del usuario, creado a partir de su conjunto de datos. Para comparar los vectores, el método más usado es la **similitud coseno**: los ángulos más pequeños entre el vector de usuario y las películas son aquellas que se le recomendarán al usuario.

### Matriz contenido

La matriz contenido es una matriz bidimensional que contiene columnas con todas las posibles características de las películas y filas con cada una de las películas. Los valores de cada característica pueden ser 0 o 1; por ejemplo, si consideramos *[Acción, Comedia, Drama, Christopher Nolan, Santiago Segura]* como las columnas de la matriz, la fila de la película *"Origen"* sería *[1, 0, 1, 1, 0]*, ya que es una película de acción y drama dirigida por Christopher Nolan, mientras que la de *"Padre no hay más que uno"* sería *[0, 1, 0, 0, 1]*.

In [2]:
import pandas as pd

df = pd.read_csv("datasets/movies_curated.csv")

Usaremos la **codificación multi-hot** con la librería MultiLabelBinarizer de sklearn para transformar los distintos géneros, directores y escritores en datos que podamos utilizar para entrenar nuestro modelo.

El método fit_transform toma columnas del conjunto de datos y crea una matriz con tantas columnas como valores tengan y crea una fila por cada película (filas del conjunto de datos original), asignando 1 y 0 según tengan o no las características, como hemos explicado en el apartado anterior.

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack, csr_matrix
import numpy as np

# Limpiamos las columnas de nulos
df['genres'] = df['genres'].replace("\\N", "").fillna("")
df['directors'] = df['directors'].replace("\\N", "").fillna("")
df['writers'] = df['writers'].replace("\\N", "").fillna("")

# Transformamos los strings separados por comas en listas
df['genres'] = df['genres'].apply(lambda x: [g.strip() for g in x.split(",") if g.strip()])
df['directors'] = df['directors'].apply(lambda x: [d.strip() for d in x.split(",") if d.strip()])
df['writers'] = df['writers'].apply(lambda x: [w.strip() for w in x.split(",") if w.strip()])

# Creamos 'cast' = directors + writers
df['cast'] = df['directors'] + df['writers']

# Con el 'sparse output' nos aseguramos que solo
# se guarden los valores distintos a 0
# Si se guardaran todos, se necesitarían cientos de
# gigas de memoria para poder procesar el conjunto de datos
mlb_genres = MultiLabelBinarizer(sparse_output=True)
mlb_cast = MultiLabelBinarizer(sparse_output=True)

# Aplicamos la codificación multi-hot
genres_matrix = mlb_genres.fit_transform(df['genres'])
cast_matrix = mlb_cast.fit_transform(df['cast'])

# Combinamos las matrices con las características de las películas
item_matrix = hstack([genres_matrix, cast_matrix]).tocsr()

# Vemos el tamaño y la densidad de la matriz creada
matrix_shape = item_matrix.shape
nonzeros = item_matrix.nnz
sparsity = 1 - (nonzeros / (matrix_shape[0] * matrix_shape[1]))

matrix_shape, nonzeros, round(sparsity, 4)


((548451, 411591), 1916655, 1.0)

Visualizamos las primeras 5 películas de la matriz que hemos creado para comprobar que se ha procesado correctamente.

Podemos comparar la información con el conjunto de datos del csv original.

In [4]:
# Guardamos los nombres de las columnas
feature_names = list(mlb_genres.classes_) + list(mlb_cast.classes_)

# Convertimos las primeras 5 filas y 20 columnas en un array
dense_sample = item_matrix[:5, :20].toarray()

# Transformamos en un conjunto para visualizarlo
sample_df = pd.DataFrame(dense_sample, 
                         columns=feature_names[:20], 
                         index=df['primaryTitle'][:5])
sample_df

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV
primaryTitle,,,,,,,,,,,,,,,,,,,,
MISS JERRY,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
THE CORBETT-FITZSIMMONS FIGHT,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
THE STORY OF THE KELLY GANG,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
THE PRODIGAL SON,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
ROBBERY UNDER ARMS,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### Vector usuario

Similar a la matriz de contenido, los gustos del usuario se almacenan en una matriz, esta vez con una sola fila (un vector), ya que solo hay un usuario. En esta fila se encuentran las características de las películas vistas por el usuario con importancia según la puntuación dada.

Para obtener las características de las películas, usamos el nombre y el año de salida de la película para encontrarlas en la matriz de contenido, este vector es multiplicado por la puntuación del usuario y sumado al vector usuario. Una vez todas las características han sido sumadas, el vector es normalizado para poder compararlo con el resto de películas.

In [ ]:
# Crea un diccionario de datos con:
# Clave: título y año de salida
# Valor: índice de la película en el conjunto de datos original
title_year_to_index = {
    (title, int(year)) : idx
    for idx, (title, year) in enumerate(zip(df['primaryTitle'], df['startYear']))
    if not pd.isna(year)
}

# Leemos el csv con el conjunto de datos del usuario
ratings_df = pd.read_csv("datasets/ratings.csv")

# Creamos feature_names para mapear las columnas
feature_names = (
    list(mlb_genres.classes_) + 
    list(mlb_cast.classes_)
)

# Identificamos qué índices corresponden a directores/escritores
cast_feature_start = len(mlb_genres.classes_)  # posición de inicio en item_matrix
cast_features = {
    name: idx + cast_feature_start
    for idx, name in enumerate(mlb_cast.classes_)
}

# Definimos el bias para directores y escritores
director_bias = 5
writer_bias = 5


# Construcción del perfil de usuario
user_profile = csr_matrix((1, item_matrix.shape[1]))

for _, row in ratings_df.iterrows():
    movie_name = row['Name'].upper()
    year = int(row['Year'])
    rating = row['Rating']

    key = (movie_name, year)
    if key in title_year_to_index:
        movie_idx = title_year_to_index[key]
        movie_vector = item_matrix[movie_idx].toarray().flatten()

        # Aplicamos bias a directores y escritores
        for d in df.iloc[movie_idx]['directors']:
            if d in cast_features:
                movie_vector[cast_features[d]] *= director_bias

        for w in df.iloc[movie_idx]['writers']:
            if w in cast_features:
                movie_vector[cast_features[w]] *= writer_bias

        # Sumamos al perfil del usuario
        user_profile += rating * csr_matrix(movie_vector)

user_vector = user_profile.toarray().flatten()

# Normalizamos
norm = np.linalg.norm(user_vector)
user_vector_normalized = user_vector / norm if norm > 0 else user_vector

# Mostrar las 20 características principales
sorted_indices = np.argsort(user_vector_normalized)[::-1] 
sorted_values = user_vector_normalized[sorted_indices]

for idx, val in zip(sorted_indices[:20], sorted_values[:20]):
    print(f"{feature_names[idx]}: {val}")

Drama: 0.3987876420134604
nm0000264: 0.3526841837646395
nm0001885: 0.2581502788380351
nm0000186: 0.22106390075144414
nm0575523: 0.1890678098532088
Comedy: 0.17408782184176227
nm0594503: 0.14834551234636384
nm0000040: 0.12362126028863653
nm0048918: 0.11998534086838251
nm0000343: 0.11998534086838251
Romance: 0.11329524913511513
nm0074311: 0.11271350202787449
nm0000876: 0.11198631814382368
nm0093081: 0.10253292765116324
nm0001068: 0.10180574376711243
nm0170043: 0.0952610888106552
nm0218840: 0.09089798550635039
nm0000419: 0.09089798550635039
nm0024622: 0.08362614666584235
nm1488800: 0.07999022724558834


Comprobamos según el csv con las películas vistas por el usuario que el vector final coincide con lo que esperábamos.



### Similitud coseno

Una vez obtenidos la matriz contenido y el vector usuario, solo nos queda comparar usando la similitud coseno:

$$\text{similarity} = \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}$$

Donde $\mathbf{A}$ es el vector del usuario y $\mathbf{B}$ es el vector de una película. Cuanto más cercana a 1 sea la puntuación, más similares son los vectores. Hay que calcular el resultado por cada película, excepto las ya vistas por el usuario.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

watched_df = pd.read_csv("datasets/watched.csv")

# Cogemos las películas vistas por el usuario
seen_movies = {
    (row['Name'].upper(), int(row['Year']))
    for _, row in watched_df.iterrows()
}

# Calculamos la similitud coseno entre el usuario y todas las películas
similarities = cosine_similarity(
    item_matrix, user_vector_normalized.reshape(1, -1)
).flatten()

# Creamos una lista con índices y similitudes, excluyendo películas ya vistas
recommendations = []
for idx, (title, year) in enumerate(zip(df['primaryTitle'], df['startYear'])):
    if pd.isna(year):  # ignorar películas sin año
        continue
    key = (title, int(year))
    if key in seen_movies:  # ignorar películas vistas
        continue
    recommendations.append((idx, similarities[idx]))

# Ordenamos por similitud descendente y tomamos las 10 primeras
top10 = sorted(recommendations, key=lambda x: x[1], reverse=True)[:10]

# Mostramos resultados
for idx, sim in top10:
    print(f"{df.iloc[idx]['primaryTitle']} ({df.iloc[idx]['startYear']}) - Similitud: {sim:.4f}")

BITTER CHRISTMAS (2026) - Similitud: 0.5314
HOLA, ¿ESTÁS SOLA? (1995) - Similitud: 0.4322
DEAR WENDY (2005) - Similitud: 0.4171
MARGARITA EN LA LUNA (2022) - Similitud: 0.4157
MA MA (2015) - Similitud: 0.4157
THE ADVENTURES OF A MADCAP (1915) - Similitud: 0.4051
WHEN LAW COMES TO HADES (1923) - Similitud: 0.4051
NO. 111 (1937) - Similitud: 0.4051
KEGLESS (1998) - Similitud: 0.4051
L.A. (2000) - Similitud: 0.4051
